## COVID Drivers: Data Quality Assessment

### Table of Contents
* [Read the Data](#read)</BR>
* [Create New Date Variables](#dates)</BR>
* [Evaluate Variables from CRASH and FLAGS](#eval)</BR>
* [Calculate Driver Counts and Compare](#drv-calc)

Import packages

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import os
import ruptures as rpt
import altair as alt

import statsmodels.api as sm

In [2]:
# Import project specific utilities
from utils.functions import *

In [3]:
path_in = 'data/aux/selected_crash_and_flags.csv'

### <a id='read'>Read the data</a>

Read dataset created in covid_drivers_02_DESC.ipynb

In [4]:
df_init = pd.read_csv(path_in, low_memory=False)

Read PERSON and VEHICLE to evaluate DRIVER_COUNT_<age> and DRIVER_<age> variables

In [5]:
person = pd.DataFrame()
vehicle = pd.DataFrame()

for folder in os.listdir(path_raw):
    #print(folder)
    year = folder[-4:]
    #print(year)
    if 'statewide' in folder.lower():
    #if year != '2020' and 'statewide' in folder.lower():
        tmp = read_data(folder, year, 'PERSON')
        person = (tmp.copy() if person.empty else person.copy() if tmp.empty
            else pd.concat([person, tmp]) # if both DataFrames non empty
        )

        tmp = read_data(folder, year, 'VEHICLE')
        vehicle = (tmp.copy() if vehicle.empty else vehicle.copy() if tmp.empty
            else pd.concat([vehicle, tmp]) # if both DataFrames non empty
        )
            

### <a id='dates'>Create New Date Variables</a>

In [6]:
df = df_init.copy()

In [7]:
df['POST_COVID'] = [1 if (i > 2020) or (i == 2020 and j >= 3) else 0 for (i, j) in zip(df['CRASH_YEAR'], df['CRASH_MONTH'])]

In [8]:
df['CRASH_MN_NAME'] = pd.to_datetime(df['CRASH_MONTH'], format='%m').dt.month_name().str[:3]

In [9]:
df['CRASH_YEAR'] = df['CRASH_YEAR'].astype('int64').astype('str')

In [10]:
df['CRASH_DATE'] = df.CRASH_YEAR.str.cat( df.CRASH_MN_NAME, sep = '-' )

In [11]:
df['CRASH_DATE'].head()

0    2005-Jan
1    2005-Jan
2    2005-Jan
3    2005-Jan
4    2005-Jan
Name: CRASH_DATE, dtype: str

In [12]:
df['CRASH_DATE'] = pd.to_datetime( df.CRASH_DATE )

/var/folders/ns/b_tjb7711nd7hm27qtmj_83w0000gn/T/ipykernel_65764/892648127.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['CRASH_DATE'] = pd.to_datetime( df.CRASH_DATE )


In [13]:
df['CRASH_DATE'].head()

0   2005-01-01
1   2005-01-01
2   2005-01-01
3   2005-01-01
4   2005-01-01
Name: CRASH_DATE, dtype: datetime64[us]

In [14]:
date_cols = ['CRASH_DATE', 
             'CRASH_MONTH', 
             'CRASH_MN_NAME', 
             'CRASH_YEAR', 
             'POST_COVID']

In [15]:
outcome = ['POST_COVID']

### <a id='eval'>Evaluate Variables from CRASH and FLAGS</a>

In [16]:
df.columns.tolist()

['COUNTY',
 'CRASH_MONTH',
 'CRASH_YEAR',
 'CRN',
 'DRIVER_COUNT_16YR',
 'DRIVER_COUNT_17YR',
 'DRIVER_COUNT_18YR',
 'DRIVER_COUNT_19YR',
 'DRIVER_COUNT_20YR',
 'DRIVER_COUNT_50_64YR',
 'DRIVER_COUNT_65_74YR',
 'DRIVER_COUNT_75PLUS',
 'URBAN_RURAL',
 'AGGRESSIVE_DRIVING',
 'DRIVER_16YR',
 'DRIVER_17YR',
 'DRIVER_18YR',
 'DRIVER_19YR',
 'DRIVER_20YR',
 'DRIVER_50_64YR',
 'DRIVER_65_74YR',
 'DRIVER_75PLUS',
 'NHTSA_AGG_DRIVING',
 'NO_CLEARANCE',
 'RUNNING_RED_LT',
 'RUNNING_STOP_SIGN',
 'SPEEDING',
 'SPEEDING_RELATED',
 'TAILGATING',
 'COUNTYx',
 'URBAN_RURALx',
 'POST_COVID',
 'CRASH_MN_NAME',
 'CRASH_DATE']

In [17]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 2461193 entries, 0 to 2461192
Data columns (total 34 columns):
 #   Column                Dtype         
---  ------                -----         
 0   COUNTY                int64         
 1   CRASH_MONTH           int64         
 2   CRASH_YEAR            str           
 3   CRN                   int64         
 4   DRIVER_COUNT_16YR     int64         
 5   DRIVER_COUNT_17YR     int64         
 6   DRIVER_COUNT_18YR     int64         
 7   DRIVER_COUNT_19YR     int64         
 8   DRIVER_COUNT_20YR     int64         
 9   DRIVER_COUNT_50_64YR  int64         
 10  DRIVER_COUNT_65_74YR  int64         
 11  DRIVER_COUNT_75PLUS   int64         
 12  URBAN_RURAL           int64         
 13  AGGRESSIVE_DRIVING    int64         
 14  DRIVER_16YR           int64         
 15  DRIVER_17YR           int64         
 16  DRIVER_18YR           int64         
 17  DRIVER_19YR           int64         
 18  DRIVER_20YR           int64         
 19  DRIVER_50_6

In [18]:
df.COUNTY.nunique()

67

In [19]:
df.COUNTY.value_counts()

COUNTY
2     236677
67    214621
46    170534
9     120646
36    111555
       ...  
47      4182
52      2672
56      1522
27      1383
12       992
Name: count, Length: 67, dtype: int64

#### Evaluate Variables from FLAGS

In [20]:
flags_cols = ['AGGRESSIVE_DRIVING',
'CRN',
'DRIVER_16YR',
'DRIVER_17YR',
'DRIVER_18YR',
'DRIVER_19YR',
'DRIVER_20YR',
'DRIVER_50_64YR',
'DRIVER_65_74YR',
'DRIVER_75PLUS',
'NHTSA_AGG_DRIVING',
'NO_CLEARANCE',
'RUNNING_RED_LT',
'RUNNING_STOP_SIGN',
'SPEEDING',
'SPEEDING_RELATED',
'TAILGATING']

In [21]:
for c in flags_cols:
    miss = df[c].isna().sum()
    pct = round((df.loc[df[c].isna()].shape[0]/df.shape[0])*100, 2)
    if miss > 0:
        print()
        print(c)
        print(str(miss) + ' missing')
        print(str(pct) + '% missing')
        print(df[c].value_counts())
        print()
    else:
        print(c + ' has no missing values')

AGGRESSIVE_DRIVING has no missing values
CRN has no missing values
DRIVER_16YR has no missing values
DRIVER_17YR has no missing values
DRIVER_18YR has no missing values
DRIVER_19YR has no missing values
DRIVER_20YR has no missing values
DRIVER_50_64YR has no missing values
DRIVER_65_74YR has no missing values
DRIVER_75PLUS has no missing values
NHTSA_AGG_DRIVING has no missing values
NO_CLEARANCE has no missing values
RUNNING_RED_LT has no missing values
RUNNING_STOP_SIGN has no missing values
SPEEDING has no missing values
SPEEDING_RELATED has no missing values
TAILGATING has no missing values


#### Evaluate month and year variables

In [22]:
date_cols = ['CRASH_MONTH', 'CRASH_YEAR']

In [23]:
for c in date_cols:
    print(c + ': ' + str(df[c].isna().sum()))

CRASH_MONTH: 0
CRASH_YEAR: 0


In [24]:
df['CRASH_YEAR'].value_counts()

CRASH_YEAR
2005    134261
2007    132152
2016    129607
2006    129253
2018    128541
2017    128441
2015    127470
2008    126184
2011    125616
2019    125452
2012    124501
2013    124366
2009    121794
2010    121612
2014    121547
2021    118100
2022    116147
2024    110813
2023    110736
2020    104600
Name: count, dtype: int64

In [25]:
for c in date_cols:
    print(c)
    print(df[c].isna().sum())
    print()

CRASH_MONTH
0

CRASH_YEAR
0



#### Evaluate location variables

In [26]:
loc_cols = ['COUNTY',
 'URBAN_RURAL']

In [27]:
for c in loc_cols:
    print(c)
    print(df[c].isna().sum())
    print()

COUNTY
0

URBAN_RURAL
0



#### Evaluate aggressive driving variables

In [28]:
agg_cols = [
'AGGRESSIVE_DRIVING',
'NHTSA_AGG_DRIVING',
'NO_CLEARANCE',
'RUNNING_RED_LT',
'RUNNING_STOP_SIGN',
'SPEEDING',
'SPEEDING_RELATED',
'TAILGATING'
]

In [29]:
for c in agg_cols:
    miss = df[c].isna().sum()
    pct = round((df.loc[df[c].isna()].shape[0]/df.shape[0])*100, 2)
    if miss > 0:
        print()
        print(c)
        print(str(miss) + ' missing')
        print(str(pct) + '% missing')
        print(df[c].value_counts())
        print()
    else:
        print(c + ' has no missing values')

AGGRESSIVE_DRIVING has no missing values
NHTSA_AGG_DRIVING has no missing values
NO_CLEARANCE has no missing values
RUNNING_RED_LT has no missing values
RUNNING_STOP_SIGN has no missing values
SPEEDING has no missing values
SPEEDING_RELATED has no missing values
TAILGATING has no missing values


### <a id='drv-calc'>Calculate Driver Counts and Compare</a>

#### Select Rows with Vehicles in Transit or Hit and Run

In [30]:
enum_dict['UNIT_TYPE']

{1: 'Motor vehicle in transport',
 2: 'Legally parked',
 3: 'Illegally parked',
 5: 'Hit and run vehicle',
 6: 'Disabled from a previous crash',
 21: 'Train',
 30: 'Non-Motorist',
 33: 'Personal Delivery Device',
 51: 'Phantom vehicle'}

In [31]:
moving = vehicle.loc[vehicle['UNIT_TYPE'].isin([1, 5])].copy()

In [32]:
select_crn = moving['CRN'].unique().tolist()

In [33]:
df2 = df.loc[df_init['CRN'].isin(select_crn)].copy()

In [34]:
df.shape[0]

2461193

In [35]:
df2.shape[0]

2460484

#### Evaluate variables from PERSON and VEHICLE

Limit to selected CRN

In [36]:
person2 = person.loc[person['CRN'].isin(select_crn)].copy()

In [37]:
vehicle2 = vehicle.loc[vehicle['CRN'].isin(select_crn)].copy()

In [38]:
enum_dict['PERSON_TYPE']

{1: 'Driver',
 2: 'Passenger',
 4: 'Non-Motorist Operator',
 5: 'Non-Motorist Occupant',
 7: 'Pedestrian',
 8: 'Other',
 9: 'Unknown'}

In [39]:
enum_dict['UNIT_TYPE']

{1: 'Motor vehicle in transport',
 2: 'Legally parked',
 3: 'Illegally parked',
 5: 'Hit and run vehicle',
 6: 'Disabled from a previous crash',
 21: 'Train',
 30: 'Non-Motorist',
 33: 'Personal Delivery Device',
 51: 'Phantom vehicle'}

In [40]:
person2['KEY'] = [str(i) + '_' + str(j) for (i, j) in zip(person2['CRN'], person2['UNIT_NUM'])]

In [41]:
person2['PERSON_TYPEx'] = [np.nan if str(x)=='nan' else enum_dict['PERSON_TYPE'][x] for x in person2['PERSON_TYPE']]

In [42]:
vehicle2['KEY'] = [str(i) + '_' + str(j) for (i, j) in zip(vehicle2['CRN'], vehicle2['UNIT_NUM'])]

In [43]:
vehicle2['UNIT_TYPEx'] = [enum_dict['UNIT_TYPE'][x] if x != 31 else x for x in vehicle2['UNIT_TYPE']]

In [44]:
per_cols = ['CRN','PERSON_TYPE','UNIT_NUM','AGE']
veh_cols = ['CRN','UNIT_NUM','UNIT_TYPE']

In [45]:
for c in per_cols:
    print(c + ': ' + str(person2[c].isna().sum()))

CRN: 0
PERSON_TYPE: 24
UNIT_NUM: 0
AGE: 0


In [46]:
for c in veh_cols:
    print(c + ': ' + str(vehicle2[c].isna().sum()))

CRN: 0
UNIT_NUM: 0
UNIT_TYPE: 0


In [47]:
person2.loc[person2['PERSON_TYPE'].isna()].shape[0]/person2.shape[0]

4.2391749576391614e-06

Missing AGE values for drivers

In [48]:
person2.loc[(person2['AGE']==99) & (person2['PERSON_TYPE']==1)].shape[0]

86941

In [49]:
person2.loc[(person2['AGE']==99) & (person2['PERSON_TYPE']==1)].shape[0]/person2.loc[(person2['PERSON_TYPE']==1)].shape[0]

0.021761439892911117

In [50]:
person2.loc[(person2['AGE']==99)]['PERSON_TYPEx'].value_counts()

PERSON_TYPEx
Driver                   86941
Passenger                78220
Unknown                   7301
Pedestrian                1485
Other                     1016
Non-Motorist Operator      418
Non-Motorist Occupant      100
Name: count, dtype: int64

In [51]:
summ_a = person2.groupby(['PERSON_TYPEx']).\
    aggregate(PERSON_TYPE_cnt=('CRN', 'count')).\
    reset_index().\
    copy()

summ_a

,PERSON_TYPEx,PERSON_TYPE_cnt
0,Driver,3995186
1,Non-Motorist Occupant,2663
2,Non-Motorist Operator,29323
3,Other,6769
4,Passenger,1535806
5,Pedestrian,80891
6,Unknown,10817


In [52]:
summ_b = person2.loc[person2['AGE']==99].\
    groupby(['PERSON_TYPEx']).\
    aggregate(PERSON_TYPE_cnt=('CRN', 'count')).\
    reset_index().\
    copy()

summ_b

,PERSON_TYPEx,PERSON_TYPE_cnt
0,Driver,86941
1,Non-Motorist Occupant,100
2,Non-Motorist Operator,418
3,Other,1016
4,Passenger,78220
5,Pedestrian,1485
6,Unknown,7301


In [53]:
summ_a['PERSON_TYPE_pct'] = round((summ_a['PERSON_TYPE_cnt']/person2.shape[0])*100, 2)

In [54]:
summ_b['PERSON_TYPE_pct'] = round((summ_b['PERSON_TYPE_cnt']/person2.loc[person2['AGE']==99].shape[0])*100, 2)

In [55]:
summ_a

,PERSON_TYPEx,PERSON_TYPE_cnt,PERSON_TYPE_pct
0,Driver,3995186,70.57
1,Non-Motorist Occupant,2663,0.05
2,Non-Motorist Operator,29323,0.52
3,Other,6769,0.12
4,Passenger,1535806,27.13
5,Pedestrian,80891,1.43
6,Unknown,10817,0.19


In [56]:
summ_b

,PERSON_TYPEx,PERSON_TYPE_cnt,PERSON_TYPE_pct
0,Driver,86941,49.54
1,Non-Motorist Occupant,100,0.06
2,Non-Motorist Operator,418,0.24
3,Other,1016,0.58
4,Passenger,78220,44.57
5,Pedestrian,1485,0.85
6,Unknown,7301,4.16


In [57]:
drv_age_ukn = person2.loc[(person2['AGE']==99) & (person2['PERSON_TYPE']==1)]['KEY'].unique().tolist()
all_drv = person2.loc[person2['PERSON_TYPE']==1]['KEY'].unique().tolist()

In [58]:
vehicle2.loc[vehicle2['KEY'].isin(all_drv)]['UNIT_TYPEx'].value_counts()

UNIT_TYPEx
Motor vehicle in transport        3913387
Hit and run vehicle                 81255
Illegally parked                      202
Legally parked                        105
Phantom vehicle                        85
Disabled from a previous crash         74
Non-Motorist                           23
Train                                  22
Name: count, dtype: int64

In [59]:
vehicle2.loc[vehicle2['KEY'].isin(drv_age_ukn)]['UNIT_TYPEx'].value_counts()

UNIT_TYPEx
Hit and run vehicle               57323
Motor vehicle in transport        29555
Phantom vehicle                      46
Legally parked                        7
Illegally parked                      3
Train                                 2
Disabled from a previous crash        2
Name: count, dtype: int64

In [60]:
summ_c = vehicle2.loc[vehicle2['KEY'].isin(all_drv)].\
    groupby(['UNIT_TYPEx']).\
    aggregate(UNIT_TYPE_cnt=('CRN', 'count')).\
    reset_index().\
    copy()

summ_c

,UNIT_TYPEx,UNIT_TYPE_cnt
0,Disabled from a previous crash,74
1,Hit and run vehicle,81255
2,Illegally parked,202
3,Legally parked,105
4,Motor vehicle in transport,3913387
5,Non-Motorist,23
6,Phantom vehicle,85
7,Train,22


In [61]:
summ_d = vehicle2.loc[vehicle2['KEY'].isin(drv_age_ukn)].\
    groupby(['UNIT_TYPEx']).\
    aggregate(UNIT_TYPE_cnt=('CRN', 'count')).\
    reset_index().\
    copy()

summ_d

,UNIT_TYPEx,UNIT_TYPE_cnt
0,Disabled from a previous crash,2
1,Hit and run vehicle,57323
2,Illegally parked,3
3,Legally parked,7
4,Motor vehicle in transport,29555
5,Phantom vehicle,46
6,Train,2


In [62]:
summ_c['UNIT_TYPE_pct'] = round((summ_c['UNIT_TYPE_cnt']/vehicle2.loc[vehicle2['KEY'].isin(all_drv)].shape[0])*100, 2)

In [63]:
summ_d['UNIT_TYPE_pct'] = round((summ_d['UNIT_TYPE_cnt']/vehicle2.loc[vehicle2['KEY'].isin(drv_age_ukn)].shape[0])*100, 2)

In [64]:
summ_c

,UNIT_TYPEx,UNIT_TYPE_cnt,UNIT_TYPE_pct
0,Disabled from a previous crash,74,0.00
1,Hit and run vehicle,81255,2.03
2,Illegally parked,202,0.01
3,Legally parked,105,0.00
4,Motor vehicle in transport,3913387,97.95
5,Non-Motorist,23,0.00
6,Phantom vehicle,85,0.00
7,Train,22,0.00


In [65]:
summ_d

,UNIT_TYPEx,UNIT_TYPE_cnt,UNIT_TYPE_pct
0,Disabled from a previous crash,2,0.00
1,Hit and run vehicle,57323,65.94
2,Illegally parked,3,0.00
3,Legally parked,7,0.01
4,Motor vehicle in transport,29555,34.00
5,Phantom vehicle,46,0.05
6,Train,2,0.00


PERSON_TYPE

In [66]:
person2.loc[person2['PERSON_TYPE'].isna(), ['CRN','UNIT_NUM','PERSON_TYPE','AGE','KEY']]

,CRN,UNIT_NUM,PERSON_TYPE,AGE,KEY
448,2013003674,1,NaN,34,2013003674_1
4497,2012111808,1,NaN,28,2012111808_1
25656,2012078509,2,NaN,99,2012078509_2
37735,2012122388,1,NaN,99,2012122388_1
37737,2012122388,3,NaN,99,2012122388_3
57531,2012125478,2,NaN,48,2012125478_2
63956,2012104261,3,NaN,99,2012104261_3
72311,2012088602,2,NaN,99,2012088602_2
79714,2012125533,2,NaN,99,2012125533_2
94341,2012098292,1,NaN,58,2012098292_1


* 2012098292_1: AGE 58
* 2012128324_2: AGE 54
* 2012120078_1: AGE 51

In [67]:
people_to_chk = person2.loc[person2['PERSON_TYPE'].isna() & ((person2['AGE']>50) & (person2['AGE']<99))]['KEY'].unique().tolist()
people_to_chk

['2012098292_1', '2012128324_2', '2012120078_1']

In [68]:
vehicle2.loc[vehicle2['KEY'].isin(people_to_chk), ['CRN', 'UNIT_NUM', 'KEY', 'UNIT_TYPEx']]

,CRN,UNIT_NUM,KEY,UNIT_TYPEx
91781,2012098292,1,2012098292_1,Motor vehicle in transport
156886,2012128324,2,2012128324_2,Motor vehicle in transport
214216,2012120078,1,2012120078_1,Motor vehicle in transport


In [69]:
person2.loc[person2['KEY'].isin(people_to_chk), ['CRN', 'UNIT_NUM', 'KEY', 'AGE', 'PERSON_TYPEx']]

,CRN,UNIT_NUM,KEY,AGE,PERSON_TYPEx
94341,2012098292,1,2012098292_1,58,NaN
183636,2012128324,2,2012128324_2,54,NaN
183637,2012128324,2,2012128324_2,27,Passenger
183638,2012128324,2,2012128324_2,49,Passenger
289560,2012120078,1,2012120078_1,99,Driver
289561,2012120078,1,2012120078_1,51,NaN
289562,2012120078,1,2012120078_1,99,Passenger


#### Limit to persons driving

In [70]:
enum_dict['PERSON_TYPE']

{1: 'Driver',
 2: 'Passenger',
 4: 'Non-Motorist Operator',
 5: 'Non-Motorist Occupant',
 7: 'Pedestrian',
 8: 'Other',
 9: 'Unknown'}

In [71]:
drivers = person2.loc[person2['PERSON_TYPE']==1, ['CRN','PERSON_TYPE', 'AGE']].copy()

In [72]:
drivers.shape[0]

3995186

In [73]:
drivers.head()

,CRN,PERSON_TYPE,AGE
0,2005066315,1.0,26
1,2005109861,1.0,17
2,2005185756,1.0,46
3,2005185756,1.0,38
4,2005015297,1.0,47


In [74]:
drivers.AGE.isna().sum()

np.int64(0)

In [75]:
drivers['TRACK'] = [1 if (x >=16 and x <= 20) or (x >=50) else 0 for x in drivers['AGE']]

In [76]:

drivers['GROUP'] = drivers['AGE']

In [77]:
drivers['GROUP'].value_counts()

GROUP
18    124235
19    122532
21    119626
20    117803
22    116074
       ...  
7         30
2         23
3         22
4         21
6         18
Name: count, Length: 99, dtype: int64

In [78]:
drivers['GROUP'] = ['No Track' if (x <= 15) or (x >= 21 and x <= 49) else 'CNT_50_64YR' if (x >= 50 and x <= 64) else 'CNT_65_74YR' if (x >= 65 and x <= 74) else 'CNT_75PLUS' if x >= 75 and x < 99 else 'CNT_' + str(x) + 'YR' for x in drivers['GROUP']]

In [79]:
drv_age_cnt = drivers.groupby(['CRN','GROUP']).\
    aggregate(DRIVER_COUNTS=('CRN','count')).\
    reset_index().\
    copy()

In [80]:
drv_age_cnt

,CRN,GROUP,DRIVER_COUNTS
0,2005000003,CNT_18YR,1
1,2005000006,CNT_19YR,1
2,2005000010,CNT_16YR,1
3,2005000012,CNT_65_74YR,1
4,2005000012,No Track,1
...,...,...,...
3376809,2025047213,No Track,1
3376810,2025047762,No Track,2
3376811,2025048164,No Track,1
3376812,2025049635,No Track,2


In [81]:
drv_age_cnt.GROUP.value_counts()

GROUP
No Track       1721618
CNT_50_64YR     669464
CNT_65_74YR     236085
CNT_75PLUS      159562
CNT_18YR        122219
CNT_19YR        120957
CNT_20YR        116311
CNT_17YR        103942
CNT_99YR         85358
CNT_16YR         41298
Name: count, dtype: int64

In [82]:
drv_age_cnt.CRN.nunique()

2451586

In [83]:
drv_age_cnt.shape[0]

3376814

In [84]:
drv_age_cnt2 = drv_age_cnt.pivot(index=['CRN'], values='DRIVER_COUNTS', columns='GROUP').reset_index().copy()

In [85]:
drv_age_cnt2

GROUP,CRN,CNT_16YR,CNT_17YR,CNT_18YR,CNT_19YR,CNT_20YR,CNT_50_64YR,CNT_65_74YR,CNT_75PLUS,CNT_99YR,No Track
0,2005000003,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2005000006,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2005000010,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2005000012,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0
4,2005000013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...
2451581,2025047213,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2451582,2025047762,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
2451583,2025048164,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2451584,2025049635,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0


In [86]:
df2.columns

Index(['COUNTY', 'CRASH_MONTH', 'CRASH_YEAR', 'CRN', 'DRIVER_COUNT_16YR',
       'DRIVER_COUNT_17YR', 'DRIVER_COUNT_18YR', 'DRIVER_COUNT_19YR',
       'DRIVER_COUNT_20YR', 'DRIVER_COUNT_50_64YR', 'DRIVER_COUNT_65_74YR',
       'DRIVER_COUNT_75PLUS', 'URBAN_RURAL', 'AGGRESSIVE_DRIVING',
       'DRIVER_16YR', 'DRIVER_17YR', 'DRIVER_18YR', 'DRIVER_19YR',
       'DRIVER_20YR', 'DRIVER_50_64YR', 'DRIVER_65_74YR', 'DRIVER_75PLUS',
       'NHTSA_AGG_DRIVING', 'NO_CLEARANCE', 'RUNNING_RED_LT',
       'RUNNING_STOP_SIGN', 'SPEEDING', 'SPEEDING_RELATED', 'TAILGATING',
       'COUNTYx', 'URBAN_RURALx', 'POST_COVID', 'CRASH_MN_NAME', 'CRASH_DATE'],
      dtype='str')

In [87]:
drv_age_cnt2.columns

Index(['CRN', 'CNT_16YR', 'CNT_17YR', 'CNT_18YR', 'CNT_19YR', 'CNT_20YR',
       'CNT_50_64YR', 'CNT_65_74YR', 'CNT_75PLUS', 'CNT_99YR', 'No Track'],
      dtype='str', name='GROUP')

In [88]:
derived_cnt = ['CNT_16YR', 
       'CNT_17YR',
       'CNT_18YR',
       'CNT_19YR',
       'CNT_20YR', 
       'CNT_50_64YR',
       'CNT_65_74YR',
       'CNT_75PLUS']

crash_cnt = ['DRIVER_COUNT_16YR', 
       'DRIVER_COUNT_17YR',
       'DRIVER_COUNT_18YR',
       'DRIVER_COUNT_19YR',
       'DRIVER_COUNT_20YR',
       'DRIVER_COUNT_50_64YR',
       'DRIVER_COUNT_65_74YR',
       'DRIVER_COUNT_75PLUS']

ages = [16, 17, 18, 19, 20, 50, 65, 75]

drv_counts = ['DRIVER_COUNT_16YR',
       'CNT_16YR', 
       'DRIVER_COUNT_17YR',
       'CNT_17YR',
       'DRIVER_COUNT_18YR', 
       'CNT_18YR',
       'DRIVER_COUNT_19YR', 
       'CNT_19YR',
       'DRIVER_COUNT_20YR',
       'CNT_20YR',
       'DRIVER_COUNT_50_64YR', 
       'CNT_50_64YR',
       'DRIVER_COUNT_65_74YR', 
       'CNT_65_74YR',
       'DRIVER_COUNT_75PLUS',
       'CNT_75PLUS']

In [89]:
drv_age_cnt3 = pd.merge(drv_age_cnt2, df2, on=['CRN'], how='left').\
    loc[:,['CRN', 'CRASH_DATE'] + drv_counts].\
    copy()

In [90]:
diff_cols = []

for i in range(0, len(ages)):
    drv_age_cnt3['DIFF_' + str(ages[i])] = [i - j for (i, j) in zip(drv_age_cnt3[crash_cnt[i]], drv_age_cnt3[derived_cnt[i]])]
    diff_cols.append('DIFF_' + str(ages[i]))
    print(diff_cols)

['DIFF_16']
['DIFF_16', 'DIFF_17']
['DIFF_16', 'DIFF_17', 'DIFF_18']
['DIFF_16', 'DIFF_17', 'DIFF_18', 'DIFF_19']
['DIFF_16', 'DIFF_17', 'DIFF_18', 'DIFF_19', 'DIFF_20']
['DIFF_16', 'DIFF_17', 'DIFF_18', 'DIFF_19', 'DIFF_20', 'DIFF_50']
['DIFF_16', 'DIFF_17', 'DIFF_18', 'DIFF_19', 'DIFF_20', 'DIFF_50', 'DIFF_65']
['DIFF_16', 'DIFF_17', 'DIFF_18', 'DIFF_19', 'DIFF_20', 'DIFF_50', 'DIFF_65', 'DIFF_75']


In [91]:
drv_age_cnt3.head()

,CRN,CRASH_DATE,DRIVER_COUNT_16YR,CNT_16YR,DRIVER_COUNT_17YR,CNT_17YR,DRIVER_COUNT_18YR,CNT_18YR,DRIVER_COUNT_19YR,CNT_19YR,...,DRIVER_COUNT_75PLUS,CNT_75PLUS,DIFF_16,DIFF_17,DIFF_18,DIFF_19,DIFF_20,DIFF_50,DIFF_65,DIFF_75
0,2005000003,2005-01-01,0,NaN,0,NaN,1,1.0,0,NaN,...,0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
1,2005000006,2005-01-01,0,NaN,0,NaN,0,NaN,1,1.0,...,0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
2,2005000010,2005-01-01,1,1.0,0,NaN,0,NaN,0,NaN,...,0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2005000012,2005-01-01,0,NaN,0,NaN,0,NaN,0,NaN,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
4,2005000013,2005-01-01,0,NaN,0,NaN,0,NaN,0,NaN,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [92]:
drv_age_cnt3['TOTAL_DIFF'] = drv_age_cnt3[diff_cols].sum(axis=1)

In [93]:
drv_age_cnt3['TOTAL_DIFF'].unique()

array([ 0., -1., -2., -3.])

In [94]:
drv_age_cnt3.loc[drv_age_cnt3['TOTAL_DIFF']!=0]

,CRN,CRASH_DATE,DRIVER_COUNT_16YR,CNT_16YR,DRIVER_COUNT_17YR,CNT_17YR,DRIVER_COUNT_18YR,CNT_18YR,DRIVER_COUNT_19YR,CNT_19YR,...,CNT_75PLUS,DIFF_16,DIFF_17,DIFF_18,DIFF_19,DIFF_20,DIFF_50,DIFF_65,DIFF_75,TOTAL_DIFF
1716,2005037307,2005-01-01,0,NaN,0,NaN,0,NaN,0,NaN,...,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,-1.0
6821,2005067402,2005-01-01,0,NaN,0,NaN,1,1.0,0,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,-1.0,NaN,NaN,-1.0
9385,2005075858,2005-01-01,0,NaN,0,NaN,0,NaN,1,1.0,...,NaN,NaN,NaN,NaN,0.0,NaN,-1.0,NaN,NaN,-1.0
13296,2005090113,2005-02-01,0,NaN,0,NaN,0,NaN,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,-1.0
14764,2005096373,2005-02-01,0,NaN,0,NaN,0,NaN,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2420236,2024089717,2024-10-01,0,NaN,0,NaN,0,NaN,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,-1.0
2422342,2024091993,2024-10-01,0,NaN,0,NaN,0,NaN,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,-1.0
2422707,2024092383,2024-09-01,0,NaN,0,NaN,0,NaN,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,-1.0
2426643,2024096641,2024-10-01,0,NaN,0,1.0,0,NaN,0,NaN,...,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,0.0,NaN,-1.0


In [95]:
drv_age_cnt3.loc[drv_age_cnt3['TOTAL_DIFF']!=0].to_csv('data/aux/age_grp_diff_by_crn.csv', index=False)

In [96]:
summ = drv_age_cnt3.groupby(['CRASH_DATE']).\
    aggregate(TOTAL_CRASHES=('CRN','count')).\
    reset_index().\
    copy()

In [97]:
summ

,CRASH_DATE,TOTAL_CRASHES
0,2005-01-01,12516
1,2005-02-01,11060
2,2005-03-01,10617
3,2005-04-01,10216
4,2005-05-01,10818
...,...,...
235,2024-08-01,8725
236,2024-09-01,8702
237,2024-10-01,10642
238,2024-11-01,10240


In [98]:
for c in drv_counts:
    summ = pd.concat([summ, summarize(drv_age_cnt3, 'CRASH_DATE', c, agg_func='sum')], axis=1).copy()

In [99]:
summ

,CRASH_DATE,TOTAL_CRASHES,DRIVER_COUNT_16YR_sum,CNT_16YR_sum,DRIVER_COUNT_17YR_sum,CNT_17YR_sum,DRIVER_COUNT_18YR_sum,CNT_18YR_sum,DRIVER_COUNT_19YR_sum,CNT_19YR_sum,DRIVER_COUNT_20YR_sum,CNT_20YR_sum,DRIVER_COUNT_50_64YR_sum,CNT_50_64YR_sum,DRIVER_COUNT_65_74YR_sum,CNT_65_74YR_sum,DRIVER_COUNT_75PLUS_sum,CNT_75PLUS_sum
0,2005-01-01,12516,275,275.0,668,668.0,747,747.0,700,700.0,638,639.0,3144,3146.0,831,831.0,657,657.0
1,2005-02-01,11060,240,240.0,613,613.0,616,616.0,591,591.0,545,545.0,2892,2898.0,674,674.0,586,586.0
2,2005-03-01,10617,259,259.0,591,591.0,691,691.0,575,575.0,569,569.0,2745,2746.0,771,772.0,566,567.0
3,2005-04-01,10216,268,268.0,598,598.0,659,660.0,576,576.0,533,533.0,2718,2720.0,755,755.0,664,664.0
4,2005-05-01,10818,299,299.0,676,676.0,765,765.0,712,712.0,604,604.0,2886,2890.0,833,833.0,764,764.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2024-08-01,8725,163,163.0,361,361.0,378,378.0,420,420.0,370,370.0,2679,2679.0,1175,1175.0,794,794.0
236,2024-09-01,8702,189,190.0,446,446.0,373,373.0,410,411.0,375,375.0,2474,2476.0,1131,1131.0,734,734.0
237,2024-10-01,10642,163,163.0,457,458.0,435,435.0,452,452.0,393,393.0,3293,3295.0,1532,1532.0,1017,1017.0
238,2024-11-01,10240,189,189.0,418,418.0,430,430.0,424,424.0,421,421.0,3054,3054.0,1341,1341.0,883,883.0


In [100]:
summ.to_csv('data/aux/age_grp_diff_by_dt.csv', index=False)